In [23]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import datetime
import pickle
import time 
%matplotlib inline

In [2]:
cd ~/Documents/SyncRecord/cleaning-addingLABEL/

/home/takeyama/Documents/SyncRecord/cleaning-addingLABEL


In [3]:
class RecordData:
    
    def __init__(self):
        print "__class__"
        # raw data
        self._AccX=np.array([])
        self._AccY=np.array([])
        self._AccZ=np.array([])
        self._GyrX=np.array([])
        self._GyrY=np.array([])
        self._GyrZ=np.array([])
        self._Columns=np.array([])
        
        # fft data
        self._fft_AccX=np.array([])
        self._fft_AccY=np.array([])
        self._fft_AccZ=np.array([])
        self._fft_GyrX=np.array([])
        self._fft_GyrY=np.array([])
        self._fft_GyrZ=np.array([])

        # power spectol data
        self._power_AccX=np.array([])
        self._power_AccY=np.array([])
        self._power_AccZ=np.array([])
        self._power_GyrX=np.array([])
        self._power_GyrY=np.array([])
        self._power_GyrZ=np.array([])
        
    def ImportCSV(self,csv_file):
        # design dataframe
        self._data = pd.read_csv(csv_file,encoding="SHIFT-JIS")
        self._data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        self._data.Time=pd.to_datetime(self._data.Time)
        self._data = pd.pivot_table(self._data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])
        
        # convert numpy.darray
        self._AccX=self._data.AccX.values
        self._AccY=self._data.AccY.values
        self._AccZ=self._data.AccZ.values
        self._GyrX=self._data.GyrX.values
        self._GyrY=self._data.GyrY.values
        self._GyrZ=self._data.GyrZ.values
        self._Time=self._data.index.to_pydatetime().astype('datetime64[ns]')
        self._Columns=['Time','AccX','AccY','AccZ','GyrX','GyrY','GyrZ']
        self._fft_col=['fft_AccX','fft_AccY','fft_AccZ','fft_GyrX','fft_GyrY','fft_GyrZ']
        self._power_col=['power_AccX','power_AccY','power_AccZ','power_GyrX','power_GyrY','power_GyrZ']
    
    def GetColumns(self):
        return self._Columns

    def ShowAllDf(self):
        print self._AccX;print self._AccY;print self._AccZ;print self._GyrX;print self._GyrY;print self._GyrZ;print self._Time
        
    def _String2Var(self,Sname):
        if(Sname=='AccX'):return self._AccX
        elif(Sname=='AccY'):return self._AccY
        elif(Sname=='AccZ'):return self._AccZ
        elif(Sname=='GyrX'):return self._GyrX
        elif(Sname=='GyrY'):return self._GyrY
        elif(Sname=='GyrZ'):return self._GyrZ
        elif(Sname=='Time'):return self._Time
        else:print 'Wrong Axis Name'
    
    def _Time2Num(self,time):
        index=np.where(test._Time==np.datetime64(time) )
        return index[0][0]    
    
    def _Query(self,Sname,rng=[]):
        data = self._String2Var(Sname)
        return data[rng[0]:rng[1]]
            
    def GetLength(self,Sname):
        data = _String2Var(Sname)
        print Sname+' length ='+str( len(data) )
            
    def ShowQuery(self,Sname,rng=[]):
        data = self._String2Var(Sname)
        print Sname+':'+str( data[rng[0]:rng[1]])
            
    def _sliding_window(self,Sname,samp, overlap):
        count =0
        s =self._Time[0]
        start=self._Time2Num(s)
        g = s+np.timedelta64(samp*10,'ms')
        goal= self._Time2Num(g)
        yield self._Query(Sname,[start,goal])
        
        add=overlap*0.01
        
        while True:
            try:
                count +=1
                s =s+np.timedelta64(samp*10,'ms')
                start=self._Time2Num(s)
                g  =s+np.timedelta64(samp*10,'ms')
                goal= self._Time2Num(g)
                yield self._Query(Sname,[start,goal])
            except StopIteration:
                print '_sliding_window StopIteration'
                break   
            except IndexError:
                print '_sliding_window IndexError'
                break   
    def _set_fft(self,Sname,array):
        #print array
        if(Sname=='fft_AccX'):   self._fft_AccX=np.append(self._fft_AccX,np.array([array]) )
        elif(Sname=='fft_AccY'): self._fft_AccY=np.append(self._fft_AccY,np.array([array]) )
        elif(Sname=='fft_AccZ'): self._fft_AccZ=np.append(self._fft_AccZ,np.array([array]) )
        elif(Sname=='fft_GyrX'): self._fft_GyrX=np.append(self._fft_GyrX,np.array([array]) )
        elif(Sname=='fft_GyrY'): self._fft_GyrY=np.append(self._fft_GyrY,np.array([array]) )
        elif(Sname=='fft_GyrZ'): self._fft_GyrZ=np.append(self._fft_GyrZ,np.array([array]) )
        else:print 'Wrong Axis Name'
            
    def _GetFFT(self,Sfft,samp):
        return np.load(p_path+'fft/'+Sfft+'_'+str(samp)+'.npz')['arr_0']
    
    def _GetPower(self,Spower,samp):
        return np.load(p_path+'power/'+Spower+'_'+str(samp)+'.npz')['arr_0']
    
    def CalcFFT(self,samp,overlap=0.5):
        for n,f in zip( self._Columns[1:],self._fft_col):
            print 'start'+n+'->'+f
            sw = self._sliding_window(n,samp,overlap)
            count=0
            while True:
                try:
                    #print 'count ='+str(count)
                    d=sw.next()
                    # fftの直流成分を除くsample/2のみ
                    self._set_fft(f,fftpack.fft(d)[1:(samp/2)+1])     
                    count = count+1
                except StopIteration:
                    print 'CalcFFTStopIteration'
                    break   
                    
        self._fft_AccX=self._fft_AccX.reshape(len(self._fft_AccX)/(samp/2),(samp/2) )
        self._fft_AccY=self._fft_AccY.reshape(len(self._fft_AccY)/(samp/2),(samp/2) )
        self._fft_AccZ=self._fft_AccZ.reshape(len(self._fft_AccZ)/(samp/2),(samp/2) )
        self._fft_GyrX=self._fft_GyrX.reshape(len(self._fft_GyrX)/(samp/2),(samp/2) )
        self._fft_GyrY=self._fft_GyrY.reshape(len(self._fft_GyrY)/(samp/2),(samp/2) )
        self._fft_GyrZ=self._fft_GyrZ.reshape(len(self._fft_GyrZ)/(samp/2),(samp/2) )
        
        np.savez(p_path+'fft/'+'fft_AccX_'+str(samp),self._fft_AccX)
        np.savez(p_path+'fft/'+'fft_AccY_'+str(samp),self._fft_AccY)
        np.savez(p_path+'fft/'+'fft_AccZ_'+str(samp),self._fft_AccZ)
        np.savez(p_path+'fft/'+'fft_GyrX_'+str(samp),self._fft_GyrX)
        np.savez(p_path+'fft/'+'fft_GyrY_'+str(samp),self._fft_GyrY)
        np.savez(p_path+'fft/'+'fft_GyrZ_'+str(samp),self._fft_GyrZ)

        
    def _power(self,fft_array):
        p=lambda x,y : np.sqrt(x**2+y**2)
        power_array=np.array([])
        for vector in fft_array:
            tmp = p( np.real(vector),np.imag(vector) )
            tmp = tmp/np.sum(tmp)
            power_array = np.append(power_array,tmp)
        return power_array
        
    def CalcPower(self,samp,overlap=0.5):
        local_fft_AccX=np.load(p_path+'fft/'+'fft_AccX_'+str(samp)+'.npz')['arr_0']
        local_fft_AccY=np.load(p_path+'fft/'+'fft_AccY_'+str(samp)+'.npz')['arr_0']
        local_fft_AccZ=np.load(p_path+'fft/'+'fft_AccZ_'+str(samp)+'.npz')['arr_0']
        local_fft_GyrX=np.load(p_path+'fft/'+'fft_GyrX_'+str(samp)+'.npz')['arr_0']
        local_fft_GyrY=np.load(p_path+'fft/'+'fft_GyrY_'+str(samp)+'.npz')['arr_0']
        local_fft_GyrZ=np.load(p_path+'fft/'+'fft_GyrZ_'+str(samp)+'.npz')['arr_0']
           
        self._power_AccX = self._power(local_fft_AccX)
        self._power_AccY = self._power(local_fft_AccY)
        self._power_AccZ = self._power(local_fft_AccZ)
        self._power_GyrX = self._power(local_fft_GyrX)
        self._power_GyrY = self._power(local_fft_GyrY)
        self._power_GyrZ = self._power(local_fft_GyrZ)

        np.savez(p_path+'power/'+'power_AccX_'+str(samp),self._power_AccX)
        np.savez(p_path+'power/'+'power_AccY_'+str(samp),self._power_AccY)
        np.savez(p_path+'power/'+'power_AccZ_'+str(samp),self._power_AccZ)
        np.savez(p_path+'power/'+'power_GyrX_'+str(samp),self._power_GyrX)
        np.savez(p_path+'power/'+'power_GyrY_'+str(samp),self._power_GyrY)
        np.savez(p_path+'power/'+'power_GyrZ_'+str(samp),self._power_GyrZ)
        
    def _KLD(self,vect1,vect2):
        f =lambda p,q : np.sum(p * np.log(p / q), axis=(p.ndim - 1))
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
    
    def CalcKLD(self,Sarray1,Sarray2,samp):
        dist1 = self._GetPower(Sarray1,samp)
        dist2 = self._GetPower(Sarray2,samp)
        
        self._kld = self._KLD(dist1,dist2)    
        np.savez(p_path+'kld/'+Sarray1+'_VS_'+Sarray2,self._kld)
        
    def ShowKLD(self,Sarray1,Sarray2,samp):
        tmp = np.load(p_path+'kld/'+Sarray1+'_VS_'+Sarray2+'.npz')['arr_0']
        print Sarray1+'_VS_'+Sarray2+'Kullback–Leibler Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        
    def _JSD(self,vect1,vect2):
        f =lambda p,q : 0.5*self._KLD(p,(0.5*(p+q)) )+0.5*self._KLD(q,(0.5*(p+q)) )
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
        
    def CalcJSD(self,Sarray1,Sarray2,samp):
        dist1 = self._GetPower(Sarray1,samp)
        dist2 = self._GetPower(Sarray2,samp)
        
        self._jsd = self._JSD(dist1,dist2)    
        np.savez(p_path+'jsd/'+Sarray1+'_VS_'+Sarray2,self._jsd)
        
    def ShowJSD(self,Sarray1,Sarray2,samp):
        tmp = np.load(p_path+'jsd/'+Sarray1+'_VS_'+Sarray2+'.npz')['arr_0']
        print Sarray1+'_VS_'+Sarray2+'Jensen-Shannon Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        
        
    def ShowFFT(self,name):
        data = self._String2Var(name)
        print data

In [4]:
west=RecordData()
west.ImportCSV('Conv-west-ags.csv')

left_hand=RecordData()
left_hand.ImportCSV('Conv-left-hand-ags.csv')

__class__
__class__


In [5]:
west._Time


array(['2016-05-27T18:06:15.748000000', '2016-05-27T18:06:15.758000000',
       '2016-05-27T18:06:15.768000000', ...,
       '2016-05-27T18:15:30.598000000', '2016-05-27T18:15:30.608000000',
       '2016-05-27T18:15:30.618000000'], dtype='datetime64[ns]')

In [6]:
left_hand._Time

array(['2016-05-27T18:06:15.696000000', '2016-05-27T18:06:15.706000000',
       '2016-05-27T18:06:15.716000000', ...,
       '2016-05-27T18:15:30.376000000', '2016-05-27T18:15:30.386000000',
       '2016-05-27T18:15:30.396000000'], dtype='datetime64[ns]')

In [7]:
label = pd.read_csv('label-data.csv')
label.columns=[u'Time',u'info1',u'info2',u'info3']
label.Time=pd.to_datetime(label.Time)

In [8]:
label

,Time,info1,info2,info3
0,2016-05-27 18:06:15.696,walking,NaN,NaN
1,2016-05-27 18:06:18.745,walking-turn,NaN,NaN
2,2016-05-27 18:06:19.570,walking,NaN,NaN
3,2016-05-27 18:06:23.588,walking-turn,NaN,NaN
4,2016-05-27 18:06:24.492,walking,NaN,NaN
5,2016-05-27 18:06:28.200,walking-turn,NaN,NaN
6,2016-05-27 18:06:29.954,walking,NaN,NaN
7,2016-05-27 18:06:33.333,walking-turn,NaN,NaN
8,2016-05-27 18:06:34.999,walking,NaN,NaN
9,2016-05-27 18:06:39.181,walking-turn,NaN,NaN


In [9]:
label_time=label.Time.values

In [10]:
label_time[0]

numpy.datetime64('2016-05-27T18:06:15.696000000')

In [11]:
label_time[-1]

numpy.datetime64('2016-05-27T18:15:22.789000000')

In [12]:
# データのインデックス　ラベルの最初から10ms追加した配列、ラベルの最後のじか
index=pd.date_range(start=label_time[0],end=label_time[-1],freq='ms').to_pydatetime().astype('datetime64[ns]')
type(index)

numpy.ndarray

In [13]:
 west._Time[0]

numpy.datetime64('2016-05-27T18:06:15.748000000')

In [14]:
index[1]

numpy.datetime64('2016-05-27T18:06:15.697000000')

In [16]:
len(west._Time)*len(index)

30086340342

In [17]:
len(index)

547094

In [18]:
len(west._Time)

54993

In [24]:
start = time.time()

clean_west=np.zeros(len(index))

for l in range( len(west._Time) ):
    for i in range( len(index) ):
        ans=index[i] - west._Time[l]
        if (np.timedelta64(-4,'ns') < ans ) & (ans > np.timedelta64(4,'ns') ):
            #print index[i]
            #print 'index number='+str(i)
            #print west._Time[0]
            continue
        clean_west[i]=west._Time[l]
            
np.savez('clean_west',clean_west)
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"

elapsed_time:129927.843744[sec]


In [26]:
129927/3600.0

36.090833333333336